In [182]:
import duckdb
import pandas as pd
import pandas_gbq

bucket_name = 'nhanes_clean'

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [5]:
%sql duckdb:///:default:

In [183]:
def generate_filename(title, extension):
    return "_".join(re.sub(r"\W+", " ", title).lower().split())

In [172]:
### GET METADATA DATAFRAME
metadata_df = pd.read_gbq(
    """SELECT *,
    replace(gcs_data_filename,'.XPT','.parquet') as parquet_filename
    FROM nhanes.nhanes_file_metadata
    WHERE dataset = 'all-continuous-nhanes'
    AND page_component != 'Limited Access'
    """,
    project_id="nhanes-genai",
    dialect="standard",
)

In [124]:
# 'Acrylamide & Glycidamide '.strip()

In [173]:
file_subset_df = metadata_df[["data_file_name","page_component"]].drop_duplicates()
file_subset_df["data_file_name"] = file_subset_df["data_file_name"].str.strip()
file_subset_df["page_component"] = file_subset_df["page_component"].str.strip()

file_subset_df.drop_duplicates(inplace=True)


In [175]:
# file_subset_df.sort_values(by=['data_file_name','page_component'])[file_subset_df['data_file_name'].str.contains('Acrylamide')]['data_file_name'].unique().tolist()


In [185]:
i = 0

con = duckdb.connect(database=':default:')

# starter_datasets = []
for data_file_name, page_component in file_subset_df.sort_values(by='data_file_name').values:
    file_df = metadata_df[(metadata_df['data_file_name'] == data_file_name) &
                          (metadata_df['page_component'].str.contains(page_component.strip()))]
    if file_df['start_year'].max() > 2015:
        i += 1
        files = [f"s3://{bucket_name}/data/{f}" for f in file_df['data_file_name'].tolist()]
        
        df = con.sql(f"SELECT * FROM read_parquet([{','.join(files)}]", union_by_name=True, filename=True)
        alias = generate_filename(data_file_name.lower() + " " + page_component.lower())
        
        update_bq_table(
                        df,
                        alias,
                        dataset="nhanes",
                        bucket="nhanes_clean",
                        truncate=True,
                        max_error=0,
                        schema=None,
                    )
        
        break
        
        # starter_datasets.append((data_file_name.strip(), page_component.strip()))
        
# print(i)
    


TypeError: sql(): incompatible function arguments. The following argument types are supported:
    1. (self: duckdb.duckdb.DuckDBPyConnection, query: str, *, alias: str = '', params: object = None) -> duckdb.duckdb.DuckDBPyRelation

Invoked with: <duckdb.duckdb.DuckDBPyConnection object at 0x7f6bab313130>, 'SELECT * FROM read_parquet([s3://nhanes_clean/data/Acculturation,s3://nhanes_clean/data/Acculturation,s3://nhanes_clean/data/Acculturation,s3://nhanes_clean/data/Acculturation,s3://nhanes_clean/data/Acculturation,s3://nhanes_clean/data/Acculturation,s3://nhanes_clean/data/Acculturation,s3://nhanes_clean/data/Acculturation,s3://nhanes_clean/data/Acculturation,s3://nhanes_clean/data/Acculturation,s3://nhanes_clean/data/Acculturation]'; kwargs: union_by_name=True, filename=True

In [177]:
file_df

,years,data_file_name,doc_file,data_file,date_published,doc_file_url,data_file_url,page_component,dataset,last_updated,start_year,end_year,published_date,gcs_doc_filename,gcs_data_filename,parquet_filename
226,2003-2004,Acculturation,ACQ_C Doc,ACQ_C Data [XPT - 633.4 KB],April 2006,https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/ACQ...,https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/ACQ...,Questionnaire,all-continuous-nhanes,2023-11-09 21:34:14.469017+00:00,2003.0,2004.0,2006-04-01 00:00:00,acculturation_questionnaire_all_continuous_nha...,acculturation_questionnaire_all_continuous_nha...,acculturation_questionnaire_all_continuous_nha...
449,2009-2010,Acculturation,ACQ_F Doc,ACQ_F Data [XPT - 296.6 KB],August 2012,https://wwwn.cdc.gov/Nchs/Nhanes/2009-2010/ACQ...,https://wwwn.cdc.gov/Nchs/Nhanes/2009-2010/ACQ...,Questionnaire,all-continuous-nhanes,2023-11-09 21:34:14.469017+00:00,2009.0,2010.0,2012-08-01 00:00:00,acculturation_questionnaire_all_continuous_nha...,acculturation_questionnaire_all_continuous_nha...,acculturation_questionnaire_all_continuous_nha...
534,2011-2012,Acculturation,ACQ_G Doc,ACQ_G Data [XPT - 321.9 KB],August 2014,https://wwwn.cdc.gov/Nchs/Nhanes/2011-2012/ACQ...,https://wwwn.cdc.gov/Nchs/Nhanes/2011-2012/ACQ...,Questionnaire,all-continuous-nhanes,2023-11-09 21:34:14.469017+00:00,2011.0,2012.0,2014-08-01 00:00:00,acculturation_questionnaire_all_continuous_nha...,acculturation_questionnaire_all_continuous_nha...,acculturation_questionnaire_all_continuous_nha...
1049,1999-2000,Acculturation,ACQ Doc,ACQ Data [XPT - 629.2 KB],June 2002,https://wwwn.cdc.gov/Nchs/Nhanes/1999-2000/ACQ...,https://wwwn.cdc.gov/Nchs/Nhanes/1999-2000/ACQ...,Questionnaire,all-continuous-nhanes,2023-11-09 21:34:14.469017+00:00,1999.0,2000.0,2002-06-01 00:00:00,acculturation_questionnaire_all_continuous_nha...,acculturation_questionnaire_all_continuous_nha...,acculturation_questionnaire_all_continuous_nha...
1077,2001-2002,Acculturation,ACQ_B Doc,ACQ_B Data [XPT - 681 KB],May 2004,https://wwwn.cdc.gov/Nchs/Nhanes/2001-2002/ACQ...,https://wwwn.cdc.gov/Nchs/Nhanes/2001-2002/ACQ...,Questionnaire,all-continuous-nhanes,2023-11-09 21:34:14.469017+00:00,2001.0,2002.0,2004-05-01 00:00:00,acculturation_questionnaire_all_continuous_nha...,acculturation_questionnaire_all_continuous_nha...,acculturation_questionnaire_all_continuous_nha...
1164,2005-2006,Acculturation,ACQ_D Doc,ACQ_D Data [XPT - 285.3 KB],March 2008,https://wwwn.cdc.gov/Nchs/Nhanes/2005-2006/ACQ...,https://wwwn.cdc.gov/Nchs/Nhanes/2005-2006/ACQ...,Questionnaire,all-continuous-nhanes,2023-11-09 21:34:14.469017+00:00,2005.0,2006.0,2008-03-01 00:00:00,acculturation_questionnaire_all_continuous_nha...,acculturation_questionnaire_all_continuous_nha...,acculturation_questionnaire_all_continuous_nha...
1187,2007-2008,Acculturation,ACQ_E Doc,ACQ_E Data [XPT - 281.6 KB],September 2009,https://wwwn.cdc.gov/Nchs/Nhanes/2007-2008/ACQ...,https://wwwn.cdc.gov/Nchs/Nhanes/2007-2008/ACQ...,Questionnaire,all-continuous-nhanes,2023-11-09 21:34:14.469017+00:00,2007.0,2008.0,2009-09-01 00:00:00,acculturation_questionnaire_all_continuous_nha...,acculturation_questionnaire_all_continuous_nha...,acculturation_questionnaire_all_continuous_nha...
1329,2013-2014,Acculturation,ACQ_H Doc,ACQ_H Data [XPT - 434.2 KB],March 2016,https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/ACQ...,https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/ACQ...,Questionnaire,all-continuous-nhanes,2023-11-09 21:34:14.469017+00:00,2013.0,2014.0,2016-03-01 00:00:00,acculturation_questionnaire_all_continuous_nha...,acculturation_questionnaire_all_continuous_nha...,acculturation_questionnaire_all_continuous_nha...
1349,2015-2016,Acculturation,ACQ_I Doc,ACQ_I Data [XPT - 423 KB],September 2017,https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/ACQ...,https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/ACQ...,Questionnaire,all-continuous-nhanes,2023-11-09 21:34:14.469017+00:00,2015.0,2016.0,2017-09-01 00:00:00,acculturation_questionnaire_all_continuous_nha...,acculturation_quest

In [169]:
starter_datasets

[('Acculturation', 'Questionnaire'),
 ('Albumin & Creatinine - Urine', 'Laboratory'),
 ('Alcohol Use', 'Questionnaire'),
 ('Alpha-1-Acid Glycoprotein - Serum (Surplus)', 'Laboratory'),
 ('Arsenic - Total - Urine', 'Laboratory'),
 ('Arsenics - Speciated - Urine', 'Laboratory'),
 ('Audiometry', 'Questionnaire'),
 ('Audiometry', 'Examination'),
 ('Audiometry - Acoustic Reflex', 'Examination'),
 ('Audiometry - Wideband Reflectance', 'Examination'),
 ('Blood Pressure & Cholesterol', 'Questionnaire'),
 ('Body Measures', 'Examination'),
 ('Cardiovascular Health', 'Questionnaire'),
 ('Cholesterol - High - Density Lipoprotein (HDL)', 'Laboratory'),
 ('Cholesterol - Low-Density Lipoproteins (LDL) & Triglycerides',
  'Laboratory'),
 ('Cholesterol - Total', 'Laboratory'),
 ('Chromium & Cobalt', 'Laboratory'),
 ('Chromium - Urine', 'Laboratory'),
 ('Complete Blood Count with 5-Part Differential', 'Laboratory'),
 ('Complete Blood Count with 5-Part Differential in Whole Blood',
  'Laboratory'),
 ('Co

In [59]:
metadata_df['gcs_data_filename'].tolist()

In [93]:
metadata_df[metadata_df['gcs_data_filename'].str.contains('audiometry_all_continuous_nhanes_1999_2000_data.parquet')]

,years,data_file_name,doc_file,data_file,date_published,doc_file_url,data_file_url,page_component,dataset,last_updated,start_year,end_year,published_date,gcs_doc_filename,gcs_data_filename,parquet_filename


In [149]:
file_df.sort_values(by=['start_year','end_year','data_file_name'], ascending=False).drop_duplicates(subset=['gcs_data_filename'])['gcs_data_filename'].tolist()



['physical_activity_monitor_raw_data_80hz_examination_all_continuous_nhanes_2013_2014_data.XPT',
 'physical_activity_monitor_raw_data_80hz_examination_all_continuous_nhanes_2011_2012_data.XPT']

,Success


In [ ]:
%%sql
output_df << SELECT * FROM read_parquet(f's3://nhanes/all-continuous-nhanes/data/audiometry_all*.parquet',
                                           union_by_name=True,
                                           filename=true);

In [44]:
output_df['filename_only'] = output_df['filename'].apply(lambda x: x.split('/')[-1])

In [54]:
# output_df['filename_only'].tolist()

In [61]:
output_df.merge(metadata_df[['start_year','end_year','last_updated','published_date','parquet_filename']],how='left',left_on='filename_only',right_on='parquet_filename')




,SEQN,AUQ130,AUQ140,AUQ150,AUQ160,AUQ170,AUQ180,AUQ190,AUQ200,AUQ210,...,AUQ490,AUQ500,AUQ510,filename,filename_only,start_year,end_year,last_updated,published_date,parquet_filename
0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,s3://nhanes/all-continuous-nhanes/data/audiome...,audiometry_all_continuous_nhanes_1999_2000_dat...,1999.0,2000.0,2023-11-07 21:25:40.087858+00:00,2005-02-01 00:00:00,audiometry_all_continuous_nhanes_1999_2000_dat...
1,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,s3://nhanes/all-continuous-nhanes/data/audiome...,audiometry_all_continuous_nhanes_1999_2000_dat...,1999.0,2000.0,2023-11-07 21:25:42.493038+00:00,2002-06-01 00:00:00,audiometry_all_continuous_nhanes_1999_2000_dat...
2,2.0,2.0,4.0,2.0,NaN,NaN,NaN,2.0,NaN,2.0,...,NaN,NaN,NaN,s3://nhanes/all-continuous-nhanes/data/audiome...,audiometry_all_continuous_nhanes_1999_2000_dat...,1999.0,2000.0,2023-11-07 21:25:40.087858+00:00,2005-02-01 00:00:00,audiometry_all_continuous_nhanes_1999_2000_dat...
3,2.0,2.0,4.0,2.0,NaN,NaN,NaN,2.0,NaN,2.0,...,NaN,NaN,NaN,s3://nhanes/all-continuous-nhanes/data/audiome...,audiometry_all_continuous_nhanes_1999_2000_dat...,1999.0,2000.0,2023-11-07 21:25:42.493038+00:00,2002-06-01 00:00:00,audiometry_all_continuous_nhanes_1999_2000_dat...
4,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,s3://nhanes/all-continuous-nhanes/data/audiome...,audiometry_all_continuous_nhanes_1999_2000_dat...,1999.0,2000.0,2023-11-07 21:25:40.087858+00:00,2005-02-01 00:00:00,audiometry_all_continuous_nhanes_1999_2000_dat...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204049,124820.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,s3://nhanes/all-continuous-nhanes/data/audiome...,audiometry_all_continuous_nhanes_2017_2020_dat...,2017.0,2020.0,2023-11-07 21:25:42.493038+00:00,2021-09-01 00:00:00,audiometry_all_continuous_nhanes_2017_2020_dat...
204050,124821.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,s3://nhanes/all-continuous-nhanes/data/audiome...,audiometry_all_continuous_nhanes_2017_2020_dat...,2017.0,2020.0,2023-11-07 21:25:40.087858+00:00,2022-03-01 00:00:00,audiometry_all_continuous_nhanes_2017_2020_dat...
204051,124821.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,s3://nhanes/all-continuous-nhanes/data/audiome...,audiometry_all_continuous_nhanes_2017_2020_dat...,2017.0,2020.0,2023-11-07 21:25:42.493038+00:00,2021-09-01 00:00:00,audiometry_all_continuous_nhanes_2017_2020_dat...
204052,124822.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,s3://nhanes/all-continuous-nhanes/data/audiome...,audiometry_all_continuous_nhanes_2017_2020_dat...,2017.0,2020.0,2023-11-07 21:25:40.087858+00:00,2022-03-01 00:00:00,audiometry_all_continuous_nhanes_2017_2020_dat...
